<h1>Charts with bokeh assignment</h1>
Download the nyc taxi data for 2016 January (see below) and prepare the following charts:

<ol>
    <li>A bokeh bar chart with day of the week (Monday, Tuesday, ...) on the x-axis and the average duration of rides on the y-axis. Make sure that the hover tool is activated and that it shows the average duration when the cursor hovers over it</li>
    <li>A bokeh interactive chart with a slider containing the hour of the day (0,1,...23) and the average total amount for each hour for each day of the week. I.e., the chart should contain days of the week on the x-axis and the mean total amount on the y-axis for a particular hour of the day. Moving the slider (e.g., from 10 to 11) should replace the chart for 1000 hrs by the chart for 1100 hrs). Don't forget the tooltip</li>
    <ul><li><a href="https://docs.bokeh.org/en/latest/docs/reference/models/widgets/sliders.html">sliders</a></li>
        <li><a href="https://docs.bokeh.org/en/latest/docs/reference/models/glyphs/vbar.html">vbar</a></li>
        <li>note that column names must be strings for converting a data frame into a column data source</li>
    </ul>
    <li>A piechart that shows how much of the total payment comes from each day of the week. The pie should have seven slices, one for each day, and the size of each slice depends on the fraction it contributes to the total. Again, don't forget the tooltip</li>
    
</ol>
<li>For the purposes of this exercise, remove any taxi rides that are less than 5 minute in duration</li>

<h2>NYC taxi data</h2>
<li>NYC taxi trip data is collected and made available (yellow, green, and black cabs)</li>
<li>We'll use data from January 2016</li>
<li><a href="https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet">https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet</a></li>
<li>The data is in <a href="https://parquet.apache.org/">parquet</a> format. Parquet is a data interchange format created by the <a href="https://www.apache.org/">Apache Foundation</a> for efficient data storage and retreival. Sort of like JSON but in binary</li>
<li>Use pandas <span style="color:blue">read_parquet</span> function to import the data</li>

<li>You may need to install pyarrow and fastparquet (using pip) - not sure!</li>

In [1]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

output_notebook()

Loading BokehJS ...

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#Get the data
datasource = "Resources/yellow_tripdata_2022-01.parquet"
df = pd.read_parquet(datasource, engine='pyarrow')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2463931 entries, 0 to 2463930
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

<span style="color:blue">Start with a small subset of the data</span>
<br>
<li>After you've completed the assignment with the subset, you can try using all the data</li>

In [3]:
df = df.sample(frac=0.2)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 492786 entries, 716827 to 718771
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   VendorID               492786 non-null  int64         
 1   tpep_pickup_datetime   492786 non-null  datetime64[ns]
 2   tpep_dropoff_datetime  492786 non-null  datetime64[ns]
 3   passenger_count        478436 non-null  float64       
 4   trip_distance          492786 non-null  float64       
 5   RatecodeID             478436 non-null  float64       
 6   store_and_fwd_flag     478436 non-null  object        
 7   PULocationID           492786 non-null  int64         
 8   DOLocationID           492786 non-null  int64         
 9   payment_type           492786 non-null  int64         
 10  fare_amount            492786 non-null  float64       
 11  extra                  492786 non-null  float64       
 12  mta_tax                492786 non-null 

<h3>Get the pickup hour (e.g., 11:20 corresponds to 11, 15:30pm corresponds to 15, etc.)</h3>

In [4]:
df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,pickup_hour
716827,2,2022-01-11 10:37:04,2022-01-11 10:58:55,1.0,4.05,1.0,N,249,237,1,16.5,0.0,0.5,3.00,0.0,0.3,22.80,2.5,0.0,10
652711,2,2022-01-10 12:48:31,2022-01-10 12:53:41,1.0,0.71,1.0,N,237,229,2,5.0,0.0,0.5,0.00,0.0,0.3,8.30,2.5,0.0,12
417120,1,2022-01-07 06:33:39,2022-01-07 06:42:11,1.0,2.20,1.0,N,137,148,1,9.0,2.5,0.5,0.00,0.0,0.3,12.30,2.5,0.0,6
310833,1,2022-01-05 16:58:45,2022-01-05 17:11:24,1.0,1.70,1.0,N,137,161,1,10.0,3.5,0.5,2.00,0.0,0.3,16.30,2.5,0.0,16
663449,2,2022-01-10 14:47:09,2022-01-10 14:50:48,1.0,0.85,1.0,N,229,237,1,5.0,0.0,0.5,2.08,0.0,0.3,10.38,2.5,0.0,14


<h3>Get the day of week (0-Monday, 1-Tuesday, ...)</h3>

In [5]:
df['day_of_week'] = df['tpep_pickup_datetime'].dt.dayofweek
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,pickup_hour,day_of_week
716827,2,2022-01-11 10:37:04,2022-01-11 10:58:55,1.0,4.05,1.0,N,249,237,1,...,0.0,0.5,3.00,0.0,0.3,22.80,2.5,0.0,10,1
652711,2,2022-01-10 12:48:31,2022-01-10 12:53:41,1.0,0.71,1.0,N,237,229,2,...,0.0,0.5,0.00,0.0,0.3,8.30,2.5,0.0,12,0
417120,1,2022-01-07 06:33:39,2022-01-07 06:42:11,1.0,2.20,1.0,N,137,148,1,...,2.5,0.5,0.00,0.0,0.3,12.30,2.5,0.0,6,4
310833,1,2022-01-05 16:58:45,2022-01-05 17:11:24,1.0,1.70,1.0,N,137,161,1,...,3.5,0.5,2.00,0.0,0.3,16.30,2.5,0.0,16,2
663449,2,2022-01-10 14:47:09,2022-01-10 14:50:48,1.0,0.85,1.0,N,229,237,1,...,0.0,0.5,2.08,0.0,0.3,10.38,2.5,0.0,14,0


<h3>Get the taxi ride duration in minutes</h3>
<li>I've done this for you</li>

In [6]:
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime'])/np.timedelta64(1, 's')/60.0
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,pickup_hour,day_of_week,duration
716827,2,2022-01-11 10:37:04,2022-01-11 10:58:55,1.0,4.05,1.0,N,249,237,1,...,0.5,3.00,0.0,0.3,22.80,2.5,0.0,10,1,21.850000
652711,2,2022-01-10 12:48:31,2022-01-10 12:53:41,1.0,0.71,1.0,N,237,229,2,...,0.5,0.00,0.0,0.3,8.30,2.5,0.0,12,0,5.166667
417120,1,2022-01-07 06:33:39,2022-01-07 06:42:11,1.0,2.20,1.0,N,137,148,1,...,0.5,0.00,0.0,0.3,12.30,2.5,0.0,6,4,8.533333
310833,1,2022-01-05 16:58:45,2022-01-05 17:11:24,1.0,1.70,1.0,N,137,161,1,...,0.5,2.00,0.0,0.3,16.30,2.5,0.0,16,2,12.650000
663449,2,2022-01-10 14:47:09,2022-01-10 14:50:48,1.0,0.85,1.0,N,229,237,1,...,0.5,2.08,0.0,0.3,10.38,2.5,0.0,14,0,3.650000


<h3>Remove rides of 5 minutes or less and save in df</h3>

In [7]:
df = df[df['duration'] > 5.0]
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,pickup_hour,day_of_week,duration
716827,2,2022-01-11 10:37:04,2022-01-11 10:58:55,1.0,4.05,1.0,N,249,237,1,...,0.5,3.0,0.0,0.3,22.8,2.5,0.0,10,1,21.850000
652711,2,2022-01-10 12:48:31,2022-01-10 12:53:41,1.0,0.71,1.0,N,237,229,2,...,0.5,0.0,0.0,0.3,8.3,2.5,0.0,12,0,5.166667
417120,1,2022-01-07 06:33:39,2022-01-07 06:42:11,1.0,2.20,1.0,N,137,148,1,...,0.5,0.0,0.0,0.3,12.3,2.5,0.0,6,4,8.533333
310833,1,2022-01-05 16:58:45,2022-01-05 17:11:24,1.0,1.70,1.0,N,137,161,1,...,0.5,2.0,0.0,0.3,16.3,2.5,0.0,16,2,12.650000
1825054,1,2022-01-24 18:52:36,2022-01-24 18:58:06,3.0,1.60,1.0,N,141,75,2,...,0.5,0.0,0.0,0.3,10.8,2.5,0.0,18,0,5.500000


<h1>PROBLEM 1: Average duration by day of week bar chart</h1>

<h3>group the data by day of week</h3>

In [8]:
day_of_week_group = df.groupby('day_of_week')

<h3>Get the mean ride duration for each group</h3>
<li>And make a df out of it</li>
<li>day_of_week_mean has the day of week as the index</li>
<li>the dataframe will have seven rows with indexes 0,1,2,..7</li>
<li>add a new column with values Monday, Tuesday, Wedensday,...,Sunday</li>

In [9]:
day_of_week_mean = day_of_week_group['duration'].mean()
day_of_week_mean_df = day_of_week_mean.to_frame()
day_of_week_mean_df['day_of_week_name'] = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_of_week_mean_df

,duration,day_of_week_name
day_of_week,,
0,15.986243,Monday
1,16.013996,Tuesday
2,16.117845,Wednesday
3,16.691915,Thursday
4,16.827248,Friday
5,16.308065,Saturday
6,16.566258,Sunday


<h3>Make a column data source object from this dataframe</h3>

In [10]:
from bokeh.models import ColumnDataSource
cdata = ColumnDataSource(day_of_week_mean_df)

<h3>Draw the vertical bar chart</h3>
<li>You must include tooltips that show the duration when hovering over a bar</li>


In [11]:
text_labels = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

tooltips = [("duration (minutes)", "@duration")]


p = figure(x_range=text_labels, y_range=(0, 20), width=900, height=400, title="Average Trip Duration by Day of Week", tooltips=tooltips)
p.vbar(x='day_of_week_name', top='duration', width=0.5, source=cdata)

p.xgrid.grid_line_color = None
p.y_range.start = 0    
show(p)

<h1>PROBLEM 2: Interactive chart with slider</h1>
<li>In this second problem, construct an interactive chart that shows the distribution of total fare amount by day of week while varying the pickup_hour</li>
<li>Each chart will have day of the week on the x-axis and the average total fare as the height of the bars for a single pickup_hour</li>
<li>Construct a slider that slides from 0 to 23 with the graph for all 24 pickup_hours</li>

<h3>Group the data by day of week and, within day of week by pickup_hour</h3>

In [12]:
hour_group = df.groupby(['day_of_week', 'pickup_hour'])

<h3>Get the average total amount for each group and unstack so that rows are weekdays (0, 1,...,7) and cols are hours (0,1,...23)</h3>
<li>Then add an additional column (24) as a copy of column 0. Col 24 will be the display column</li>
<li>Finally, convert all column names into str (since pickup_hour is an int and column data source objects need str column names)</li>
<li>amount_df should like like (col names should be strings):</li>
<li>Note that your numbers may be different if you're using a random subset of the data</li>

<pre>
	0	1	2	3	4	5	6	7	8	9	...	16	17	18	19	20	21	22	23	24	dayname
day_of_week																					
0	28.519591	27.871129	21.032270	22.854089	27.553843	27.676799	22.630954	19.790608	18.589532	18.314011	...	19.823463	19.087813	19.056134	19.880450	20.452326	22.545119	23.010316	25.220471	28.519591	Monday
1	26.523835	24.473547	22.464758	25.709178	24.027132	23.652944	21.546370	18.771057	17.414492	17.255911	...	19.631683	19.094055	18.343164	19.008278	19.145718	19.704968	20.285164	21.180154	26.523835	Tuesday
2	22.662570	23.111039	23.067922	19.263433	25.915858	25.043071	19.286858	17.697268	17.354702	16.875423	...	20.199947	18.939048	18.146021	18.688651	18.839771	18.879133	19.636418	19.631235	22.662570	Wednesday
3	20.806747	20.891364	20.104057	21.230155	21.545217	23.838166	19.245900	17.484051	17.593239	17.560638	...	19.601307	19.309099	18.675074	19.065926	18.602721	18.435254	18.848939	18.878703	20.806747	Thursday
4	19.091578	18.271015	19.781767	19.620808	23.030823	25.265687	21.332188	19.119613	18.374634	18.916849	...	19.979930	19.077043	18.743151	18.467401	17.985403	17.955496	17.998007	18.500657	19.091578	Friday
5	18.792271	18.033738	18.594487	19.076232	20.591734	23.261181	27.161993	21.153212	19.545850	17.098222	...	18.881910	18.955416	18.305366	17.865027	18.519547	19.021029	19.285096	18.937453	18.792271	Saturday
6	18.807702	18.348061	18.054653	19.275509	20.891784	28.260720	27.280063	23.220415	21.592732	18.765725	...	20.551586	20.630724	19.809704	20.566784	21.532079	22.483101	24.575294	27.071233	18.807702	Sunday
</pre>

In [13]:
# Get the average total amount for each group and 
# unstack so that rows are weekdays (0, 1,...,7) and cols are hours (0,1,...23)
amount_df = hour_group['duration'].mean().unstack()
# Then add an additional column (24) as a copy of column 0. 
amount_df[24] = amount_df[0]
# Col 24 will be the display column
amount_df['display'] = amount_df[24]
# convert all column names into strings
amount_df.columns = amount_df.columns.astype(str)
amount_df

pickup_hour,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,display
day_of_week,,,,,,,,,,,,,,,,,,,,,
0,19.733233,17.007490,15.660309,18.062145,23.257471,20.831981,16.465702,17.158148,16.247239,16.396910,...,17.115923,15.639485,14.822789,14.572874,15.457269,14.554528,15.934105,15.372395,19.733233,19.733233
1,18.366255,25.876784,31.109004,15.393838,16.700964,16.374940,18.519662,15.350039,17.422289,14.345327,...,16.584892,15.207647,15.027253,14.727898,13.750880,15.027202,14.164768,15.536454,18.366255,18.366255
2,18.114809,25.941771,21.229983,26.919835,16.656711,17.051094,16.385055,15.870373,17.140333,15.486047,...,16.539294,16.379084,15.415145,14.873615,16.100667,13.498688,14.581057,15.555237,18.114809,18.114809
3,20.249232,33.295267,13.608593,24.471135,14.908117,16.882720,15.146228,15.305938,16.961859,17.006670,...,20.252722,16.062227,14.738487,14.417894,15.260814,14.870741,14.593629,14.820369,20.249232,20.249232
4,14.551246,14.294412,14.435764,13.907536,15.805048,17.156070,18.320311,18.506991,17.706789,17.187422,...,19.116811,18.323812,15.506503,14.969245,15.442102,15.111393,14.216093,13.930464,14.551246,14.551246
5,18.454070,16.706631,15.978775,15.612695,20.750050,18.567595,16.490598,15.674916,18.798431,13.589645,...,16.400722,17.691752,16.602304,16.268442,16.493936,15.130147,17.024471,16.449470,18.454070,18.454070
6,15.074916,17.801792,16.323794,16.787718,16.945320,17.046942,22.380226,16.467443,15.290434,15.899168,...,17.680718,16.322364,17.431545,17.715334,17.407027,17.796763,18.315143,17.563613,15.074916,15.074916


<h3>Draw the interactive chart by filling in the code below</h3>
<li>Mostly done. You need to fill in the missing parts identified by ??)</li>

In [14]:
from bokeh.models import Slider, CustomJS, CustomJSTickFormatter
from bokeh.layouts import row

source = ColumnDataSource(amount_df)

#Average Total Fare. Note the formatting so that the values
# show up currency formatted
tooltips = [
    ("Average Total Fare", "$@display{0,0.00}"),
]

p = figure(height=400, 
           width=600,
           x_axis_label = "Day of Week",
           y_axis_label = "Average Total Fare",
           title="Chart",
           tooltips=tooltips,
           min_border_left = 50,
           min_border_right = 50)

p.vbar(x='day_of_week', top='display', source=source, width=0.9,
      fill_color='red', line_color='black',fill_alpha = 0.75,
      hover_fill_alpha = 1.0, hover_fill_color = 'navy')

p.xgrid.grid_line_color = None
p.y_range.start = 0    

# Set tick positions 
p.xaxis.ticker = [0,1,2,3,4,5,6]

# Set tick labels
p.xaxis.major_label_overrides = {
    0: 'Monday',
    1: 'Tuesday',
    2: 'Wednesday',
    3: 'Thursday',
    4: 'Friday',
    5: 'Saturday',
    6: 'Sunday'
}

# Set axis labels
p.xaxis.formatter = CustomJSTickFormatter(code="""
    var labels = %s;
    return labels[tick];
""" % text_labels)

slider = Slider(start=0, end=24, value=0, step=1, title="Pickup Hour")


jscallback = CustomJS(args={'source':source,'slider':slider},code="""
        console.log(' changed selected option', slider.value);

        var data = source.data;
        var col = String(slider.value);
        console.log(' changed selected option', col);
        data['display'] = data[col];

        source.change.emit();
""")


slider.js_on_change('value', jscallback)

layout = row(p,slider)
show(layout)

<h1>PROBLEM 3: Piechart</h1>
<li>Use the total_amount column</li>
<li>Use the grouped by day of week data</li>
<li>Sum the total amount for each group and then compute the fractional amount for each day</li>
<li>Using the class notebook piechart as a guide, construct the piechart for distribution of total amount collected by day of week</li>

In [15]:
from bokeh.palettes import Turbo256
from bokeh.transform import cumsum

total_amount_df = day_of_week_group['total_amount'].sum().to_frame()

total_amount_df['day_of_week_name'] = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

total_amount_df['pct'] = ((total_amount_df['total_amount'] / total_amount_df['total_amount'].sum()) * 100.0).round(2)

total_amount_df['angle'] = total_amount_df['pct'] / (total_amount_df['pct'].sum()) * 2 * np.pi

import random 
# Shuffle the Turbo256 palette
shuffled_colors = random.sample(Turbo256, len(Turbo256))

# Select the first n colors from the shuffled palette
colors = shuffled_colors[:len(total_amount_df)]
total_amount_df['color'] = colors

tooltips = [
    ("Day of Week", "@day_of_week_name"),
    ("Total Amount", "$@total_amount{0,0.00}"),
    ("Percent", "@pct{0.00}%")
]

cdata = ColumnDataSource(total_amount_df)

p = figure(height=350, 
           title="Total Amount by Day of Week", 
           tools="hover",
           tooltips=tooltips,
           x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='day_of_week_name', source=cdata)

p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None

show(p)